In [17]:
import datetime
import time
import requests
import xmltodict
import json

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select

<h1>Lê a lista de IMOs através de um arquivo .txt</h1>

In [60]:
imolist = []

with open('imos.txt', 'r') as file:
    for line in file:
        imolist.append(line.strip())

print(imolist)

['9376359', '9384318', '9384796', '9437907', '9448580', '9461116', '9492414', '9513440', '9527025', '9533074', '9565699', '9567453', '9574066', '9594509', '9650860', '9705885', '9720483', '9727663', '9732589', '9793923', '9872171', '9883493', '9928918', '9941415']


<h1>Através da API, pega os dados dos respectivos navios e cria um arquivo .txt</h1>

In [61]:
current_time = datetime.datetime.now().strftime('%H%M%S') # pega os dados do tempo atual para usá-los na criação do .txt com infos
requests.packages.urllib3.disable_warnings() # desabilita avisos chatos de SSL
# acesso à api do MarineTraffic (inserir sua chave)
api_url = 'https://services.marinetraffic.com/api/vesselmasterdata/{suachave}'
ships_data = []

for imo in imolist:
    params = {'V': 5, 'imo': imo, 'protocol': 'xml', 'msgtype': 'simple'}
    try:
        response = requests.get(api_url, params=params, verify=False)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
            print(f'Request error occurred: {err}')
    else:
        data = xmltodict.parse(response.text)
        data_dict = {
            'imo': data['MASTERDATA']['vessel']['@IMO'],
            'name': data['MASTERDATA']['vessel']['@NAME'],
            'callsign': data['MASTERDATA']['vessel']['@CALLSIGN'],
            'flag': data['MASTERDATA']['vessel']['@FLAG'],
            'vessel_type': data['MASTERDATA']['vessel']['@VESSEL_TYPE'],
            'build': data['MASTERDATA']['vessel']['@BUILD'],
            'gross_tonnage': data['MASTERDATA']['vessel']['@GROSS_TONNAGE'] + '00',
            'dwt': data['MASTERDATA']['vessel']['@SUMMER_DWT'] + '00'
        }
        ships_data.append(data_dict)

for ship in ships_data:
    print(ship)
    
with open(f'{current_time}_ships.txt', 'w') as file:
    for i, dictionary in enumerate(ships_data):
        if i > 0:
            file.write(',\n')
        json.dump(dictionary, file)
    file.write('\n')

{'imo': '9376359', 'name': 'MARIELENA', 'callsign': 'A8MN6', 'flag': 'LR', 'vessel_type': 'BULK CARRIER', 'build': '2008', 'gross_tonnage': '4274700', 'dwt': '8135400'}
{'imo': '9384318', 'name': 'PIA', 'callsign': 'V2CP3', 'flag': 'AG', 'vessel_type': 'GENERAL CARGO', 'build': '2007', 'gross_tonnage': '961800', 'dwt': '1273200'}
{'imo': '9384796', 'name': 'ASTERIS', 'callsign': 'V7LN3', 'flag': 'MH', 'vessel_type': 'BULK CARRIER', 'build': '2007', 'gross_tonnage': '3126100', 'dwt': '5362900'}
{'imo': '9437907', 'name': 'GRANDE MAROCCO', 'callsign': 'IBOQ', 'flag': 'IT', 'vessel_type': 'RO-RO/CONTAINER CARRIER', 'build': '2010', 'gross_tonnage': '4763500', 'dwt': '2572500'}
{'imo': '9448580', 'name': 'ROYAL', 'callsign': 'ICMF', 'flag': 'IT', 'vessel_type': 'BULK CARRIER', 'build': '2010', 'gross_tonnage': '4798400', 'dwt': '8733400'}
{'imo': '9461116', 'name': 'CHORUS', 'callsign': '3EZZ7', 'flag': 'PA', 'vessel_type': 'BULK CARRIER', 'build': '2011', 'gross_tonnage': '4301200', 'dwt'

In [62]:
class Navio:
    def __init__(self, imo, nome, callSign, bandeira, tipo, ano, arqueacao, dwt):
        self.imo = imo
        self.nome = nome
        self.callSign = callSign
        self.bandeira = bandeira
        self.tipo = tipo
        self.ano = ano
        self.arqueacao = arqueacao
        self.dwt = dwt

<h1>Leitura do .txt e transformação em objetos</h1>

In [63]:
with open(f'{current_time}_ships.txt', 'r') as file:
    # adiciona vírgula no final de cada objeto JSON e coloca todos os objetos dentro de uma lista
    data = '[' + file.read().replace('}\n{', '},\n{') + ']'
    navios_data = json.loads(data)

# cria uma lista de objetos Navio
navios = []
for navio_data in navios_data:
    navio = Navio(
        imo=navio_data['imo'],
        nome=navio_data['name'],
        callSign=navio_data['callsign'],
        bandeira=navio_data['flag'],
        tipo=navio_data['vessel_type'],
        ano=navio_data['build'],
        arqueacao=navio_data['gross_tonnage'],
        dwt=navio_data['dwt']
    )
    navios.append(navio)

# imprime os objetos Navio
for navio in navios:
    print(navio.__dict__)

{'imo': '9376359', 'nome': 'MARIELENA', 'callSign': 'A8MN6', 'bandeira': 'LR', 'tipo': 'BULK CARRIER', 'ano': '2008', 'arqueacao': '4274700', 'dwt': '8135400'}
{'imo': '9384318', 'nome': 'PIA', 'callSign': 'V2CP3', 'bandeira': 'AG', 'tipo': 'GENERAL CARGO', 'ano': '2007', 'arqueacao': '961800', 'dwt': '1273200'}
{'imo': '9384796', 'nome': 'ASTERIS', 'callSign': 'V7LN3', 'bandeira': 'MH', 'tipo': 'BULK CARRIER', 'ano': '2007', 'arqueacao': '3126100', 'dwt': '5362900'}
{'imo': '9437907', 'nome': 'GRANDE MAROCCO', 'callSign': 'IBOQ', 'bandeira': 'IT', 'tipo': 'RO-RO/CONTAINER CARRIER', 'ano': '2010', 'arqueacao': '4763500', 'dwt': '2572500'}
{'imo': '9448580', 'nome': 'ROYAL', 'callSign': 'ICMF', 'bandeira': 'IT', 'tipo': 'BULK CARRIER', 'ano': '2010', 'arqueacao': '4798400', 'dwt': '8733400'}
{'imo': '9461116', 'nome': 'CHORUS', 'callSign': '3EZZ7', 'bandeira': 'PA', 'tipo': 'BULK CARRIER', 'ano': '2011', 'arqueacao': '4301200', 'dwt': '8218100'}
{'imo': '9492414', 'nome': 'SUPRA ONIKI',

<h1>Realiza cadastros no site da Antaq</h1>

In [69]:
servico = Service(ChromeDriverManager().install())
email = 'dcampos@brssz.com'

for navio in navios:
    driver = webdriver.Chrome(service=servico)
    # entra no site da antaq e preenche imo
    driver.get("https://web3.antaq.gov.br/SAMA/Embarcacao/Consultar.aspx")
    time.sleep(5)
    driver.find_element('xpath', '//*[@id="txtIMO"]').click()
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(navio.imo)
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(Keys.RETURN)

    # verifica se o popup de navio já cadastrado aparece
    if (len(driver.find_elements('xpath', '//*[@id="msgCadastrar"]'))):
        driver.find_element('xpath', '//*[@id="btnCadastrarSim"]').click()

    # verifica se navio já está cadastrado
    if (len(driver.find_elements('xpath', '//*[@id="Formulario"]/fieldset/div/b[1]/u'))):
        navios.remove(navio)
        continue

    #driver.get(f"https://web3.antaq.gov.br/SAMA/Embarcacao/Cadastro.aspx?Novo={imo}")
    time.sleep(5)

    # preenchimento de dados
    driver.find_element('xpath', '//*[@id="txtNome"]').click()
    driver.find_element('xpath', '//*[@id="txtNome"]').clear()
    driver.find_element('xpath', '//*[@id="txtNome"]').send_keys(navio.nome)
    driver.find_element('xpath', '//*[@id="txtIRIN"]').clear()
    driver.find_element('xpath', '//*[@id="txtIRIN"]').send_keys(navio.callSign)
    driver.find_element('xpath', '//*[@id="ddlBandeira"]').click()
    select = Select(driver.find_element('xpath','//*[@id="ddlBandeira"]')).select_by_value(navio.bandeira)

    # verifica o tipo de navio e carga
    driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]').click()
    if (navio.tipo == 'BULK CARRIER'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[19]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[9]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'BULKER'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[19]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[9]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'GENERAL CARGO'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[10]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[4]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'TANKER'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[19]').click()
        driver.find_element('xpath', '//*[@id="ddlClasse"]/option[2]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[11]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'OIL TANKER' or navio.tipo == 'CRUDE OIL TANKER'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[31]').click()
        # opção de petróleo não aparecendo
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'PASSENGER SHIP'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[28]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[15]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'VEHICLES CARRIER'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[35]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[17]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'CONTAINER SHIP'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[32]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[6]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'OIL/CHEMICAL TANKER'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[33]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/select[1]/option[7]').click()
        driver.find_element('xpath', '//*[@id="vncTipoCarga"]/div/input[3]').click()
    elif (navio.tipo == 'VEHICLES CARRIER'):
        driver.find_element('xpath', '//*[@id="ddlTipoEmbarcacao"]/option[33]').click()
    else:
        print("Navio não está nas opções comuns.")

    # separando cada caractere das strings arqueacao e dwt devido a um bug no formulário da ANTAQ
    navio.arqueacao = (" ".join(navio.arqueacao))
    navio.dwt = (" ".join(navio.dwt))
        
    # preenchendo dados de ano, arqueação e dwt
    driver.find_element('xpath', '//*[@id="txtAno"]').clear()
    driver.find_element('xpath', '//*[@id="txtAno"]').send_keys(navio.ano)
    driver.find_element('xpath', '//*[@id="txtArqueacao"]').clear()
    driver.find_element('xpath', '//*[@id="txtArqueacao"]').send_keys(navio.arqueacao)
    driver.find_element('xpath', '//*[@id="txtTPB"]').clear()
    driver.find_element('xpath', '//*[@id="txtTPB"]').send_keys(navio.dwt)

    # selecionando checkbox de cabotagem e longo curso
    if not (driver.find_element('xpath', '//*[@id="chkCabotagem"]').is_selected()):
        driver.find_element('xpath', '//*[@id="chkCabotagem"]').click()
        time.sleep(2)
    if not (driver.find_element('xpath', '//*[@id="chkLongoCurso"]').is_selected()):
        driver.find_element('xpath', '//*[@id="chkLongoCurso"]').click()

    # inserindo email
    driver.find_element('xpath', '//*[@id="txtEmail"]').send_keys(email)
    driver.find_element('xpath', '//*[@id="txtConfirmarEmail"]').send_keys(email)

    driver.find_element('xpath', '//*[@id="rdbSDP"]').click()

    print(f"IMO: {navio.imo}\nNome: {navio.nome}\nCallSign: {navio.callSign}\nBandeira: {navio.bandeira}\nTipo: {navio.tipo}\nAno: {navio.ano}\nArqueação: {navio.arqueacao}\nDWT: {navio.dwt}\n")
    keyword = input("enter a character or press enter to continue")
    navios.remove(navio)
    time.sleep(3)


IMO: 9567453
Nome: SERENITAS N
CallSign: V7VS8
Bandeira: MH
Tipo: BULK CARRIER
Ano: 2011
Arqueação: 3 3 0 3 2 0 0
DWT: 5 7 0 0 0 0 0

enter a character or press enter to continue
IMO: 9793923
Nome: SEASPAN HARRIER
CallSign: VRRQ8
Bandeira: HK
Tipo: CONTAINER SHIP
Ano: 2018
Arqueação: 1 1 3 8 2 8 0 0
DWT: 1 3 2 7 8 8 0 0

enter a character or press enter to continue
IMO: 9941415
Nome: DA YU
CallSign: 9V7287
Bandeira: SG
Tipo: BULK CARRIER
Ano: 2022
Arqueação: 4 9 5 4 1 0 0
DWT: 8 5 1 8 0 0 0

enter a character or press enter to continue


<h1>(Alternativa à API) Criação de uma lista dos dados dos navios através do site MarineTraffic</h1>

In [88]:
navios = []
servico = Service(ChromeDriverManager().install())

for imo in imolist:    
    driver = webdriver.Chrome(service=servico)
    # entra no site e aguarda carregar
    driver.get("https://www.marinetraffic.com/en/global-search/")
    # espera até aviso aparecer para continuar e aceita os cookies
    try:
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')))
        driver.find_element('xpath', '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
        time.sleep(2)
    finally:
        # digita IMO e pesquisa
        driver.find_element('xpath', "//*[@id='app']/div/div[2]/div[2]/header/div/div[1]/div/div[2]/div/div/input").send_keys(imo)
        driver.find_element('xpath', "//*[@id='app']/div/div[2]/div[2]/header/div/div[1]/div/div[2]/div/div/input").send_keys(Keys.RETURN)
        # clica no primeiro resultado

        time.sleep(2)
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,"//*[@id='app']/div/div[2]/div[2]/div/div/a[1]/div")))
        driver.find_element('xpath',"//*[@id='app']/div/div[2]/div[2]/div/div/a[1]/div").click()
        time.sleep(3)

        # scroll até a área onde os dados estão localizados
        driver.execute_script("scroll(0, 3000)")
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="General-content"]/div')))

        time.sleep(4)
        # coleta dos dados
        nome = driver.find_element('xpath', '//*[@id="shipName"]/b').text
        callSign = driver.find_element('xpath', '//*[@id="callSign"]/b').text
        bandeira = driver.find_element('xpath', '//*[@id="flag"]/b').text
        tipo = driver.find_element('xpath', '//*[@id="shipTypeSpecific"]/b').text
        ano = driver.find_element('xpath', '//*[@id="yearBuilt"]/b').text
        arqueacao = driver.find_element('xpath', '//*[@id="grossTonnage"]/b').text
        dwt = driver.find_element('xpath', '//*[@id="summerDwt"]/b').text

        # tratamento dos dados
        bandeira = bandeira[-3] + bandeira[-2] # pegando apenas a sigla da bandeira
        arqueacao += '00'
        dwt = dwt.replace(' t', '')
        dwt += '00'

        # separando cada caractere das strings arqueacao e dwt devido a um bug no formulário da ANTAQ
        arqueacao = (" ".join(arqueacao))
        dwt = (" ".join(dwt))

        navios.append(Navio(imo, nome, callSign, bandeira, tipo, ano, arqueacao, dwt))

        print(f"Nome: {nome},\nIMO: {imo},\nCallSign: {callSign},\nBandeira: {bandeira},\nTipo: {tipo},\nAno: {ano},\nArqueação: {arqueacao},\nDWT: {dwt}\n")
        time.sleep(3)
        driver.close()

        


Nome: MARIELENA,
IMO: 9376359,
CallSign: A8MN6,
Bandeira: LR,
Tipo: Bulk Carrier,
Ano: 2008,
Arqueação: 4 2 7 4 7 0 0,
DWT: 8 1 3 5 4 0 0

Nome: PIA,
IMO: 9384318,
CallSign: V2CP3,
Bandeira: AG,
Tipo: General Cargo,
Ano: 2007,
Arqueação: 9 6 1 8 0 0,
DWT: 1 2 7 3 2 0 0

Nome: ASTERIS,
IMO: 9384796,
CallSign: V7LN3,
Bandeira: MH,
Tipo: Bulk Carrier,
Ano: 2007,
Arqueação: 3 1 2 6 1 0 0,
DWT: 5 3 6 2 9 0 0

Nome: GRANDE MAROCCO,
IMO: 9437907,
CallSign: IBOQ,
Bandeira: IT,
Tipo: Ro-Ro/Container Carrier,
Ano: 2010,
Arqueação: 4 7 6 3 5 0 0,
DWT: 2 5 7 2 5 0 0

Nome: ROYAL,
IMO: 9448580,
CallSign: ICMF,
Bandeira: IT,
Tipo: Bulk Carrier,
Ano: 2010,
Arqueação: 4 7 9 8 4 0 0,
DWT: 8 7 3 3 4 0 0

Nome: CHORUS,
IMO: 9461116,
CallSign: 3EZZ7,
Bandeira: PA,
Tipo: Bulk Carrier,
Ano: 2011,
Arqueação: 4 3 0 1 2 0 0,
DWT: 8 2 1 8 1 0 0

Nome: SUPRA ONIKI,
IMO: 9492414,
CallSign: V7A4686,
Bandeira: MH,
Tipo: Bulk Carrier,
Ano: 2010,
Arqueação: 3 3 0 4 4 0 0,
DWT: 5 7 0 2 2 0 0

Nome: DAN CISNE,
IMO: 95

<h1>Percorre a lista de IMOs para checar manualmente a integridade do cadastro</h1>

In [71]:
imolist = [
    '9231377',
    '9274460',
    '9288576',
    '9336907',
    '9346823',
    '9376359',
    '9384318',
    '9384796',
    '9437907',
    '9448580',
    '9461116',
    '9492414',
    '9513440',
    '9527025',
    '9533074',
    '9565699',
    '9567453',
    '9574066',
    '9594509',
    '9650860',
    '9705885',
    '9720483',
    '9727663',
    '9732589',
    '9793923',
    '9872171',
    '9883493',
    '9928918',
    '9941415'
]

servico = Service(ChromeDriverManager().install())

for imo in imolist:
    driver = webdriver.Chrome(service=servico)
    # entra no site e aguarda carregar
    driver.get(f"https://web3.antaq.gov.br/SAMA/Embarcacao/Consultar.aspx")
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="txtIMO"]').click()
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(imo)
    driver.find_element('xpath', '//*[@id="txtIMO"]').send_keys(Keys.RETURN)
    time.sleep(2)